# LLM LoRA fine-tuning

⚠️ **IMPORTANT:** Before running this notebook, copy it into your user directory at `user/<your-username>/`.  
This ensures that any output or intermediate checkpoints are stored in your personal workspace.

Once copied, follow the environment setup instructions in the [README.md](./README.md), and connect this notebook to the `.venv` environment you created during setup.

In [1]:
from finetune import (
    CONFIGS,
    get_device,
    load_and_prepare_dataset,
    tokenize_dataset,
    create_lora_model,
    run_experiment,
)

/storage/home/hcoda1/6/asuresh85/dsgt-arc/fall-2025-interest-group-projects/project/01-llm-lora/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# choose config
config = CONFIGS["quick"]  # or "full"
device = get_device()
print(f"Running LoRA fine-tuning on {config['dataset_name']} ({config['model_name']})")

Running LoRA fine-tuning on rotten_tomatoes (roberta-base)


In [3]:
# load and tokenize data
dataset, text_field = load_and_prepare_dataset(config["dataset_name"])
tokenized_dataset, tokenizer = tokenize_dataset(
    dataset, text_field, config["model_name"], config["max_length"]
)

In [4]:
# create and train LoRA model
lora_alpha = config["lora_rank"] * config["lora_alpha_ratio"]
model = create_lora_model(
    config["model_name"],
    config["num_labels"],
    rank=config["lora_rank"],
    alpha=lora_alpha,
)

results, model = run_experiment(
    model=model,
    device=device,
    tokenized_dataset=tokenized_dataset,
    config=config,
    experiment_name="LoRA Fine-Tuning",
    learning_rate=config["learning_rate_lora"],
    is_full_finetuning=False,
)

print("\n=== LoRA Fine-Tuning Results ===")
for k, v in results.items():
    print(f"{k:20s}: {v}")

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There are adapters available but none are activated for the forward pass.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.342200,0.337258,0.863039
2,0.289900,0.317991,0.873358
3,0.259200,0.323877,0.877111



=== LoRA Fine-Tuning Results ===
experiment_name     : LoRA Fine-Tuning
training_time       : 80.67294120788574
train_loss          : 0.34326045652304993
eval_loss           : 0.32387739419937134
eval_accuracy       : 0.8771106941838649
total_params        : 126248795
trainable_params    : 1603163
trainable_percentage: 1.269844199305031


In [ ]:
# region
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

def tune_model(config_params):
    for k,v in config_params.items():
        config[k] = v
    lora_alpha = config["lora_rank"] * config["lora_alpha_ratio"]
    model = create_lora_model(
        config["model_name"],
        config["num_labels"],
        rank=config["lora_rank"],
        alpha=lora_alpha,
    )

    results, model = run_experiment(
        model=model,
        device=device,
        tokenized_dataset=tokenized_dataset,
        config=config,
        experiment_name="LoRA Fine-Tuning",
        learning_rate=config["learning_rate_lora"],
        is_full_finetuning=False,
    )

   
    print(f"params ======== {config_params}")
    print(f"\n=== LoRA Fine-Tuning Results ===")
    for k, v in results.items():
        print(f"{k:20s}: {v}")
    return {"loss" : -(results["eval_accuracy"]), "accuracy": results["eval_accuracy"], "status": STATUS_OK}

search_space = {
    "lora_rank" : hp.choice("lora_rank",[8, 16, 32, 64, 128]),
    "lora_alpha_ratio" : hp.choice("lora_alpha_ratio", [2,3,4,5,6,7,8]),
    "learning_rate_lora" : hp.uniform("learning_rate_lora", 1e-5,  1e-3),
    "num_epochs" : hp.choice("num_epochs",[2,3,4,5]),
    "batch_size" : hp.choice("batch_size",[16,32,64,128]),
}

search_space_2 = {
    "lora_rank" : hp.choice("lora_rank",[8, 16, 32,48, 64]),
    "lora_alpha_ratio" : hp.choice("lora_alpha_ratio", [1, 2, 3, 4]),
    "learning_rate_lora" : hp.uniform("learning_rate_lora", 4e-4,  1e-3),
    "num_epochs" : hp.choice("num_epochs",[3, 4, 5]),
    "batch_size" : hp.choice("batch_size",[8,16,32,64]),
}
trials = Trials()
best_params = fmin(
    fn=tune_model,
    # space=search_space,
    space=search_space_2,
    algo=tpe.suggest,
    max_evals=15,
    trials=trials,
    verbose=True,
)
# Find and log best results
best_trial = min(trials.results, key=lambda x: x["loss"])
actual_best_params = space_eval(search_space, best_params)
print(f"Actual best_params = {actual_best_params}")
# endregion

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There are adapters available but none are activated for the forward pass.

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.320400,0.328698,0.881801
2,0.247400,0.377371,0.878049


params ======== {'batch_size': 16, 'learning_rate_lora': 0.000613240754004354, 'lora_alpha_ratio': 2, 'lora_rank': 16, 'num_epochs': 3}
  7%|▋         | 1/15 [00:54<12:36, 54.02s/trial, best loss: -0.8780487804878049]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There are adapters available but none are activated for the forward pass.

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.328400,0.308686,0.873358
2,0.254400,0.381112,0.878987


params ======== {'batch_size': 16, 'learning_rate_lora': 0.0006919976048763512, 'lora_alpha_ratio': 1, 'lora_rank': 64, 'num_epochs': 3}
 93%|█████████▎| 14/15 [22:30<01:42, 102.10s/trial, best loss: -0.8930581613508443]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There are adapters available but none are activated for the forward pass.

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.343100,0.350343,0.848030
2,0.271200,0.304159,0.887430
3,0.216500,0.325938,0.894934


params ======== {'batch_size': 32, 'learning_rate_lora': 0.000744184792707988, 'lora_alpha_ratio': 3, 'lora_rank': 4, 'num_epochs': 4}
100%|██████████| 15/15 [23:28<00:00, 93.93s/trial, best loss: -0.8949343339587242] 
Actual best_params = {'batch_size': 64, 'learning_rate_lora': 0.000744184792707988, 'lora_alpha_ratio': 3, 'lora_rank': 16, 'num_epochs': 3}


In [ ]:
config["batch_size"]=64
config["learning_rate_lora"] = 0.00074418
config['lora_alpha_ratio'] = 3
config["lora_rank"] = 16
config['num_epochs'] = 3
tune_model(config)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There are adapters available but none are activated for the forward pass.
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.493700,0.335308,0.860225
2,0.303000,0.306515,0.883677
3,0.197200,0.294156,0.891182
4,0.150300,0.330841,0.890244


params ======== {'dataset_name': 'rotten_tomatoes', 'model_name': 'roberta-base', 'max_length': 128, 'num_labels': 2, 'num_epochs': 4, 'batch_size': 64, 'learning_rate_full': 2e-05, 'learning_rate_lora': 0.00074418, 'lora_rank': 16, 'lora_alpha_ratio': 3, 'description': 'Quick experiment: Rotten Tomatoes + RoBERTa-base'}


{'loss': -0.8902439024390244, 'accuracy': 0.8902439024390244, 'status': 'ok'}